In [9]:
import gym
import random
import os
import numpy as np
import tensorflow
from tensorflow.keras.optimizers import Adam 
from keras.models import Sequential 
from keras.layers import Dense
from collections import deque

In [10]:
env = gym.make("CartPole-v0")
env = gym.wrappers.Monitor(env,"recording",force=True)

In [17]:
class DQNAgent: 
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.epsilon = 1.0
        self.gamma = 0.95
        self.epsilon_decay = 0.995 
        
        self.epsilon_min = 0.01 
        
        self.learning_rate = 0.001 
        self.model = self._build_model()
    
    def _build_model(self): 
        model = Sequential()
        model.add(Dense(24,input_dim = self.state_size,activation='relu'))
        model.add(Dense(24,activation='relu'))
        model.add(Dense(self.action_size,activation='linear'))
        model.compile(loss='mse', optimizer = Adam(lr= self.learning_rate))
        return model 
    
    def remeber(self,state, action, reward, next_state,done):
        self.memory.append((state,action,reward,next_state,done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon: 
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
        
    def replay(self,batch_size):
        minibatch = random.sample(self.memory,batch_size)
        for state,action,reward,next_state,done in minibatch: 
            target = reward #gehen mal davon aus wir sind am ende! dann kennen wir den reward 
            #falls wir noch nicht am ende sind müssen wir den zukünftigen reward schätzen und zum aktuellen dazu rechnen
            if not done: 
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0])) 
            
            
            target_f = self.model.predict(state) 
            #print(target_f.shape)
            target_f[0][action]= target
            #update single target row. 
            self.model.fit(state,target_f,epochs=1,verbose=0) #model updaten mit einzelnem korrigierten wert
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def load(self,name):
        self.model.load_weights(name)
    
    def save(self,name):
        self.model.save_weights(name)

In [19]:
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size,action_size)
agent.load("model_output/weights550/checkpoint")

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for model_output/weights550/checkpoint

In [13]:
done = False
score=0
observation = env.reset()
while not done:
    #env.render()
    action = agent.act(observation)
    observation_, reward, done, info = env.step(action)
    score += reward

    #agent.store_transition(observation, action, reward, observation_, done)
    #agent.remeber(observation,action,reward,observation_,done)
    
    #agent.learn()
    observation = observation_

print(f'Score is:{score}')

Score is:20.0


In [14]:
env.close()